[![PyBind11](pybind11.png)](http://pybind11.readthedocs.io/en/stable/)

* Similar to Boost::Python, but easier to build
* Pure C++11 (no new language required), no dependencies
* Builds remain simple and don't require preprocessing
* Easy to customize result
* Great Gitter community
* Used in [GooFit 2.1+](https://goofit.github.io) for CUDA too [[CHEP talk]](https://indico.cern.ch/event/587955/contributions/2938087/)
* Used in boost-histogram, SciPy, PyTorch, Google, and more.
* Big updates to build system in 2.6, great developers ;)

Downsides:
* Still verbose (but highly customized interface is likely worth it in most cases)

In [ ]:
%%writefile SimpleClass.hpp
#pragma once

class Simple {
    int x;
    
public:

    Simple(int x): x(x) {}
    
    int get() const {
        return x;
    }
    
};

In [ ]:
%%writefile pybindclass.cpp

#include <pybind11/pybind11.h>
#include "SimpleClass.hpp"

namespace py = pybind11;

PYBIND11_MODULE(pybindclass, m) {
    py::class_<Simple>(m, "Simple")
        .def(py::init<int>())
        .def("get", &Simple::get)
    ;
}

UNIX only! Next section will include Windows compatible builds.

In [ ]:
import os
import sys
from pybind11 import get_include

inc = "-I " + get_include()
plat = "-undefined dynamic_lookup" if "darwin" in sys.platform else "-fPIC"
pyinc = !python3-config --cflags

print(f"{inc = }\n")
print(f"{plat = }\n")
print(f"{pyinc.s = }\n")

In [ ]:
!c++ -std=c++11 pybindclass.cpp -shared {inc} {pyinc.s} -o pybindclass.so {plat}

In [ ]:
import pybindclass

x = pybindclass.Simple(4)
x.get()

## More binding situations

Let's take a look at other binding situations.

In [ ]:
%%writefile VectorClass.hpp
#pragma once

class Vector2D {
    double x;
    double y;
    
public:

    Vector2D(double x, double y): x(x), y(y) {}
    
    float get_x() const {
        return x;
    }
    
    float get_y() const {
        return y;
    }
    
    void set_x(float val) {
        x = val;
    }
    
    void set_y(float val) {
        y = val;
    }

    
    Vector2D& operator+= (const Vector2D& other) {
        x += other.x;
        y += other.y;
        return *this;
    }
    
    Vector2D operator+ (const Vector2D& other) const {
        return Vector2D(x + other.x, y + other.y);
    }
};

The binding code for this will show a few more features than before.

In [ ]:
%%writefile vectorclass.cpp

#include <pybind11/pybind11.h>
#include <pybind11/operators.h>
#include "VectorClass.hpp"

namespace py = pybind11;
using namespace pybind11::literals;

PYBIND11_MODULE(vectorclass, m) {
    py::class_<Vector2D>(m, "Vector2D")
        .def(py::init<double, double>(), "x"_a, "y"_a)
        .def_property("x", &Vector2D::get_x, &Vector2D::set_x)
        .def_property("y", &Vector2D::get_y, &Vector2D::set_y)
        .def(py::self += py::self)
        .def(py::self + py::self)
        .def("__repr__", [](py::object self){
            return py::str("{0.__class__.__name__}({0.x}, {0.y})").format(self);
        })
    ;
}

Important / new points:

* Initializers are really easy, even if overloaded. Just use `py::init<...>()`.
* You can specify argument names with `"..."_a` and `using namespace pybind11::literals`.
* Properties allow you to specify a get and set function.
* The pybind11/operators.h header lets you describe operations with py::self
* You can manually add special functions
* There doesn't have to be a method; you can bind a lambda function as well.
* You can use a class instance _OR_ a py::object, and you can cast between them with py::cast (not shown).
* Many python classes are provided, like py::str, with common methods, like .format.
* You can access any python method with .attr (not shown).

Feel free to play with this example. Most importantly, remember to restart the notebook, because Python caches imports (which we've seen before).

In [ ]:
!c++ -std=c++11 vectorclass.cpp -shared {inc} {pyinc.s} -o vectorclass.so {plat}

In [ ]:
import vectorclass

v = vectorclass.Vector2D(1, 2)
print(f"{v.x = }, {v.y = }")

In [ ]:
v

In [ ]:
v + v

In [ ]:
vectorclass.Vector2D(x=2, y=4)

Pybind11 has fantastic documentation, and supports a lot of situations.

* Embedded mode (runs in reverse, calling Python from C++)
* Smart pointers support
* Variant support (C++17 or custom variants like from Boost)
* NumPy support (_without_ requiring NumPy during the compile!)
* Eigen support (built-in extension)
* Several external extensions provided (JSON, Abisel, and more)

Design details:

* Functions make a call-chain to handle overloads. This means there is a _little_ more overhead in calling a function than some other systems.
* The focus of the library is making _small_ extensions, if faster == bigger, it probably won't fly.